In [0]:
# !pip install pyLDAvis

In [0]:
import gzip
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import Word2Vec, LdaMulticore

from gensim.corpora import Dictionary

import numpy as np
import pyLDAvis
from pyLDAvis import gensim



In [0]:
# cd '/content/drive/My Drive/Colab Notebooks/data'

/content/drive/My Drive/Colab Notebooks/data


In [0]:
# ls

reviews_data.txt


In [0]:
def preprocesing(sentence):
  return [word for word in simple_preprocess(sentence) \
          if word not in STOPWORDS]

def read_sentences(filename):
  with open(filename, 'rb') as f:
    for line in f:
      yield preprocesing(line)



In [0]:
sentences = list(read_sentences('reviews_data.txt'))

In [0]:
len(sentences)

255404

In [0]:
%time model = Word2Vec(sentences, size=100, window=5, min_count=2)

CPU times: user 4min 31s, sys: 853 ms, total: 4min 32s
Wall time: 2min 27s


In [0]:
# Reprezentacja wektorowa

In [0]:
model.wv.most_similar('good')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('decent', 0.8384097814559937),
 ('great', 0.8179704546928406),
 ('excellent', 0.7994639873504639),
 ('ok', 0.6614031195640564),
 ('fair', 0.6350531578063965),
 ('reasonable', 0.628515362739563),
 ('average', 0.6100230813026428),
 ('nice', 0.6018450856208801),
 ('fantastic', 0.5989974737167358),
 ('terrific', 0.5949693322181702)]

In [0]:
model.wv.most_similar('bad')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('terrible', 0.7305161952972412),
 ('horrible', 0.6856048107147217),
 ('awful', 0.6574721932411194),
 ('poor', 0.611802875995636),
 ('negative', 0.5950594544410706),
 ('ok', 0.5720013380050659),
 ('okay', 0.5709905028343201),
 ('disliked', 0.5634517669677734),
 ('fair', 0.5620459914207458),
 ('unpleasant', 0.5572897791862488)]

In [0]:
model.wv.most_similar(positive=['bad', 'terrible'], negative=['good'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('horrible', 0.8429305553436279),
 ('awful', 0.8065128326416016),
 ('horrid', 0.7143263816833496),
 ('shocking', 0.7060668468475342),
 ('horrendous', 0.7013221383094788),
 ('dreadful', 0.6900390386581421),
 ('appalling', 0.6633679866790771),
 ('horrific', 0.6607958078384399),
 ('disgusting', 0.6585890054702759),
 ('nasty', 0.6542385816574097)]

In [0]:
model.wv.most_similar('bmw')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('chauffeur', 0.8780323266983032),
 ('audi', 0.8629295825958252),
 ('mercedes', 0.8555797338485718),
 ('maybach', 0.8351262807846069),
 ('sedan', 0.8215503096580505),
 ('limousine', 0.8130462169647217),
 ('lexus', 0.7826153635978699),
 ('benz', 0.780434250831604),
 ('chauffer', 0.7799681425094604),
 ('customs', 0.7766056060791016)]

In [0]:
# Modelowanie tematu - LDA

In [0]:
sentences_light = np.random.permutation(sentences)

In [0]:
sentences_light = sentences_light[:1000]

In [0]:
%time dictionary = Dictionary(sentences_light)

CPU times: user 83.6 ms, sys: 2 ms, total: 85.6 ms
Wall time: 88.5 ms


In [0]:
len(dictionary)

8855

In [0]:
%time bow_corpus = [dictionary.doc2bow(sent) for sent in sentences_light]

CPU times: user 67.4 ms, sys: 4.01 ms, total: 71.4 ms
Wall time: 72.8 ms


In [0]:
dictionary.doc2bow(['car', 'ok']) # liczymy ile razy dane słowo występuje w zdaniach. kolejnośc odwrotna, tj. ok - 393 i car - 465

[(393, 1), (465, 1)]

In [0]:
dictionary.token2id['car']

465

In [0]:
%time lda_model = LdaMulticore(bow_corpus, id2word=dictionary, num_topics=100, passes=20, workers=8)

Streaming output truncated to the last 5000 lines.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Us

CPU times: user 1min 18s, sys: 33 s, total: 1min 51s
Wall time: 1min 28s


In [0]:
for idx, topic in lda_model.print_topics(-1):
  print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.054*"good" + 0.045*"hotel" + 0.018*"room" + 0.017*"value" + 0.017*"location" + 0.016*"rooms" + 0.013*"small" + 0.013*"service" + 0.013*"price" + 0.012*"stay"
Topic: 1 
Words: 0.026*"hotel" + 0.020*"ottimo" + 0.020*"montreal" + 0.019*"jul" + 0.013*"aug" + 0.012*"old" + 0.012*"location" + 0.012*"breakfast" + 0.007*"business" + 0.007*"adatto"
Topic: 2 
Words: 0.032*"hotel" + 0.015*"th" + 0.014*"great" + 0.012*"amp" + 0.011*"room" + 0.010*"quot" + 0.009*"rooms" + 0.009*"subway" + 0.008*"blocks" + 0.008*"location"
Topic: 3 
Words: 0.033*"room" + 0.032*"hotel" + 0.014*"great" + 0.011*"quot" + 0.008*"location" + 0.008*"stay" + 0.008*"rooms" + 0.007*"nice" + 0.007*"clean" + 0.007*"breakfast"
Topic: 4 
Words: 0.035*"hotel" + 0.016*"great" + 0.014*"room" + 0.011*"small" + 0.011*"restaurant" + 0.011*"bar" + 0.010*"staff" + 0.010*"location" + 0.009*"complimentary" + 0.009*"food"
Topic: 5 
Words: 0.031*"hotel" + 0.019*"small" + 0.019*"great" + 0.011*"rooms" + 0.009*"service" + 0.

In [0]:
# Wizualizacja tematów

In [0]:
lda_vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(lda_vis)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
